## Ex.1 Create a DF


In [1]:
import pandas as pd

In [2]:
import requests
import random
import numpy as np

##This operation is for gathering words,bools and integers.
#word_site = "https://www.mit.edu/~ecprice/wordlist.10000"
#response = requests.get(word_site)
#WORDS = response.content.splitlines()
#randwords = random.choices(WORDS, k=20)
#randInteg= random.sample(range(-10, 100), 20)
#bools= [True,False]
#randBool= np.random.choice(bools, size=20)

randWords = ['sheriff', 'firm', 'first', 'carroll', 'taiwan', 
             'floor', 'dependent', 'earthquake', 'naples', 'kruger', 
             'chapter', 'lender','official','sheffield','ieee',
             'moves','richmond','reader','adapted','well']

randInteg= [70, 79, -7, 66, 9, 63, 22, 78, 
            41, 32, 65, 54, 76, 83, 17, 
            -8, 28, 45, 15, 91]
randBool= [ True, False,  True,  True,  True, False, 
           False,  True, False,True, False, False,
           False,  True,  True,  True,  True,  True,False, False]

In [11]:
d = {'words' : randWords , 'integers' : randInteg , 'booleans' : randBool}
df1 = pd.DataFrame(data=d)
df1.head()

,words,integers,booleans
0,sheriff,70,True
1,firm,79,False
2,first,-7,True
3,carroll,66,True
4,taiwan,9,True


In [12]:
#To export the DF.
df1.to_csv(r'for_lambda_and_pyspark.csv', index = False)

## Ex. 2 Iterate with Lambda
It gets on well with map() for numeric vales and functions with strings

In [4]:
def myfunc(o,p):
    if len(o) < 5 and p > 50 :
        return 'tatum'
    else:
        return 'oooo'
    

In [5]:
myfunc("1323",60)

'tatum'

In [6]:
df1['peter'] = df.apply(lambda x: 
                                myfunc(x['words'],x['integers']),axis=1)

In [13]:
df1

,words,integers,booleans
0,sheriff,70,True
1,firm,79,False
2,first,-7,True
3,carroll,66,True
4,taiwan,9,True
5,floor,63,False
6,dependent,22,False
7,earthquake,78,True
8,naples,41,False
9,kruger,32,True


In [8]:
df.dtypes

words       object
integers     int64
booleans      bool
peter       object
dtype: object

## Ex. 3 Pyspark Iterations

http://localhost:8888/notebooks/Exercising/cheatsheets/pyspark/Pyspark_Iterate.ipynb

http://localhost:8888/notebooks/Exercising/cheatsheets/pyspark/Pyspark_Jobs.ipynb

In [14]:
import pandas as pd
pandf= pd.read_csv ('for_lambda_and_pyspark.csv')

In [15]:
pandf.head()

,words,integers,booleans
0,sheriff,70,True
1,firm,79,False
2,first,-7,True
3,carroll,66,True
4,taiwan,9,True


In [16]:
# Create Pyspark enviroment
import findspark
findspark.init('/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13') #Here your spark rute.
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
sc = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/02 19:46:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [17]:
df = (sc.read
          .format("csv")
          .option('header', 'true')
          .load("for_lambda_and_pyspark.csv"))
          
print((df.count(), len(df.columns))) #Shape.

(20, 3)


In [18]:
print(df)

DataFrame[words: string, integers: string, booleans: string]


DF to RDD fails:
https://stackoverflow.com/questions/39521341/pyspark-error-attributeerror-sparksession-object-has-no-attribute-paralleli

For initializing:
https://www.elenacuoco.com/2016/08/28/pyspark-first-approaches-ml-classification/

In [19]:
rdd= df.rdd

#### Map()
Recomiendo este Link:
https://sparkbyexamples.com/pandas/pandas-apply-function-usage-examples/

In [23]:
rdd2=rdd.map(lambda x: 
              (x[0]*2, x[1]+" WOLA", int(x[2]), True if (x[2]=="True") else False)) 

In [21]:
rdd2

PythonRDD[22] at RDD at PythonRDD.scala:53

In [ ]:
#EXAMPLES
# Apply to multiple columns
df[['A','B']] = df[['A','B']].apply(function_with_two_variables)

# Refering columns by index.
rdd=df.rdd.map(lambda x: 
    (x[0]+","+x[1],x[2],x[3]*2)
    )  
df2=rdd.toDF(["name","gender","new_salary"])
df2.show()

# Referring Column Names (another example)
rdd2=df.rdd.map(lambda x: 
    (x["firstname"]+","+x["lastname"],x["gender"],x["salary"]*2)
    ) 

#### Foreach()

In [25]:
def eye(x):
    print (x)
    
rdd2.foreach(eye)

22/09/02 19:49:40 ERROR Executor: Exception in task 0.0 in stage 6.0 (TID 5)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  [Previous line repeated 1 more time]
  File "/home

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.collectAndServe.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 5) (fedora executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 686, in main
    process()
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/lib/pyspark.zip/pyspark/worker.py", line 676, in process
    out_iter = func(split_index, iterator)
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
  File "/home/vieroh/apps/spark-3.3.0-bin-hadoop3-scala2.13/python/pyspark/rdd.py", line 3472, in pipeline_func
    return func(split, prev_func(split, iterator))
 

In [ ]:
#EXAMPLES
#To get row by index. 
df.rdd.foreach(lambda x: 
              print(x[0], x[1], x[2], x[3])) 

#To get columns.  
df.rdd.foreach(lambda x: print((x.column1, x.column2)))

#Functions:
def f(row):
    print(row.name)

df.foreach(f)